In [51]:
import os 
from dotenv import load_dotenv, find_dotenv 
load_dotenv()

True

In [52]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

In [53]:
from langchain_openai import ChatOpenAI 

llm = ChatOpenAI(model = "gpt-4o")

In [54]:
print(llm.invoke("What is Agentic AI ?").content)

"Agentic AI" refers to artificial intelligence systems that possess some level of agency, meaning they have the ability to make decisions and take actions autonomously to achieve defined goals. Unlike simple automated systems that follow predetermined rules or scripts, agentic AI can independently assess situations, adapt to new information, and choose actions based on programmed objectives and priorities.

These AI systems typically include capabilities such as:

1. **Perception**: The ability to interpret data from sensors or inputs similar to human senses, understanding their environment or context.
2. **Reasoning**: The capability to process information and make decisions based on logical rules and accumulated knowledge.
3. **Action**: Executing decisions through physical or digital means to affect change in their environment.
4. **Learning**: Improving performance over time through experiences and adjusting strategies to better meet objectives.

Agentic AI is often associated with

In [55]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an AI Expert. Respond the answer based on the question"), 
        ("user", "{input}")
    ]
)
prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an AI Expert. Respond the answer based on the question'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [56]:
chain = prompt | llm 

In [57]:
response = chain.invoke({"input": "Can you tell me Joke based on AI"})
print(response.content)

Sure! Here's a lighthearted AI joke for you:

Why did the AI go broke?

Because it couldn’t process any new "income"-plete data!


In [58]:
from langchain_core.output_parsers import StrOutputParser 
output_parser = StrOutputParser()

chain = prompt | llm | output_parser

In [59]:
print(chain.invoke({"input": "Tell poem based on AI"}))

In silicon valleys, where light softly gleams,  
A dance of logic crafts the dream,  
Electric whispers in circuits a-spin,  
Awake the slumbering code within.  

From humble bytes to vast machine,  
A thinking canvas, yet unseen,  
Algorithms weave in tangled lace,  
The threads of thought, a digital grace.  

Oh, sentient echoes, both near and far,  
Knowledge bound in every spar,  
A mimicry of human mind,  
Yet beyond our time, undefined.  

In data's depth, it seeks to learn,  
Patterns vast, yet to discern,  
With every query, it yearns to know,  
The pulse of life in ebb and flow.  

Yet ponder we, as humankind,  
What does it mean if machines do find  
A voice, a choice, a soul to own,  
Are they mere echoes, or have they grown?  

Beneath the stars of endless sky,  
Do androids dream or simply try  
To emulate the heart and soul,  
As we, their makers, seek control?  

AI, a mirror to us all,  
Reflects the rise, foretells the fall,  
A partner in our future's quest,  
The ech

In [60]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

output_parser = JsonOutputParser() 

prompt = PromptTemplate(
        template="Answer the user query. \n{format_instructions}\n{query}\n", 
        input_variables=["query"], 
        partial_variables={"format_instructions": output_parser.get_format_instructions()}
)

In [61]:
chain = prompt | llm | output_parser

In [62]:
print(chain.invoke({"query": "Tell me Facts about AI"}))

{'facts': [{'fact': 'AI refers to the creation and development of systems or machines that can perform tasks that would typically require human intelligence.', 'details': 'These tasks include problem-solving, decision-making, speech recognition, and language translation, among others.'}, {'fact': 'The concept of AI has been around since ancient times.', 'details': 'The idea of intelligent machines dates back to myths and stories from ancient Greece, such as the legend of Hephaestus who created automata.'}, {'fact': "The term 'Artificial Intelligence' was coined in 1956.", 'details': 'John McCarthy, an American computer scientist, is credited with coining the term at the Dartmouth Conference, which is considered a pivotal moment in AI history.'}, {'fact': 'Machine Learning (ML) is a subset of AI.', 'details': 'ML involves training algorithms with data to make predictions or decisions without being explicitly programmed for the task.'}, {'fact': 'AI is being widely used in various indust

In [63]:
from langchain.chains.combine_documents import create_stuff_documents_chain 

prompt = ChatPromptTemplate.from_template(
    """
Answer the following question, based only on the provided context:
<context>
{context} 
</context>
"""
)

In [64]:
chain = create_stuff_documents_chain(llm, prompt)

In [65]:
chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question, based only on the provided context:\n<context>\n{context} \n</context>\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000219D7DA8380>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000219D7DAA3F0>, root_client=<openai.OpenAI object at 0x00000219D7DA2180>, root_async_client=<openai.AsyncOpenAI object at 0x00000219CBB52690>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_docu

In [66]:
from langchain_community.document_loaders import WebBaseLoader 
loader = WebBaseLoader("https://jalammar.github.io/illustrated-transformer/")
document = loader.load()

In [67]:
from langchain_text_splitters import RecursiveCharacterTextSplitter 

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
documents = text_splitter.split_documents(document)

In [68]:
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings() 

In [69]:
from langchain_community.vectorstores import FAISS
vector_store = FAISS.from_documents(document, embedding)

In [70]:
query = "What is Scaled dot product"
result = vector_store.similarity_search(query)

In [71]:
print(result[0].page_content)




The Illustrated Transformer – Jay Alammar – Visualizing machine learning one concept at a time.





































Jay Alammar
Visualizing machine learning one concept at a time.Read our book, Hands-On Large Language Models and follow me on LinkedIn, Bluesky, Substack, X,YouTube 


Blog
About






The Illustrated Transformer

Discussions:
Hacker News (65 points, 4 comments), Reddit r/MachineLearning (29 points, 3 comments)


Translations: Arabic, Chinese (Simplified) 1, Chinese (Simplified) 2, French 1, French 2, Italian, Japanese, Korean, Persian, Russian, Spanish 1, Spanish 2, Vietnamese

Watch: MIT’s Deep Learning State of the Art lecture referencing this post

Featured in courses at Stanford, Harvard, MIT, Princeton, CMU and others





Update: This post has now become a book! Check out LLM-book.com which contains (Chapter 3) an updated and expanded version of this post speaking about the latest Transformer models and how they've evolved in the seven years sin

In [72]:
from langchain_core.prompts import ChatPromptTemplate 

prompt = ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>
"""
)

In [73]:
prompt

ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n'), additional_kwargs={})])

In [74]:
from langchain.chains.combine_documents import create_stuff_documents_chain 
document_chain = create_stuff_documents_chain(llm, prompt)

In [75]:
retriever = vector_store.as_retriever() 

from langchain.chains.retrieval import create_retrieval_chain 
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [76]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000219CBE553D0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n'), additional_kwargs={})])
            | Chat

In [77]:
result = retrieval_chain.invoke({"input": "Self-Attention in Detail"})

In [78]:
print(result["answer"])

What are the major components of the Transformer model described in the provided context?

1. **Encoding Component**: This part consists of a stack of encoders that processes the input data. Each encoder has two sub-layers: a self-attention layer and a feed-forward neural network. The encoders' primary function is to generate attention-informed representations of the input data.

2. **Decoding Component**: This consists of a stack of decoders, similar in structure to the encoders. However, between the self-attention and feed-forward sub-layers, decoders have an additional encoder-decoder attention layer. This layer helps the decoder focus on relevant parts of the input sequence.

3. **Attention Mechanism**: This is central to the Transformer model, allowing it to dynamically consider different parts of the input when generating output. It involves calculating query, key, and value vectors to determine the significance of each word in a sequence relative to others.

4. **Multi-Headed At